In [1]:
import pandas as pd
import sqlite3 as sql
import os
import time
from gutenberg.cleanup import strip_headers
import urllib
import json

In [2]:
df = pd.read_hdf('/home/jon/Code/gitenberg-scrape/pg-metadata-rdf.hdf').T

In [3]:
hasAuthor = df[df.author.notnull()]

In [6]:
james = hasAuthor[hasAuthor.author.str.contains('James, Henry')]

In [4]:
victorians = ['Eliot, George', 'Stevenson, Robert Louis', 'Hardy, Thomas', 'James, Henry', 'Conrad, Joseph',
             'Ward, Humphry']

In [5]:
def isVictorian(author): 
    for victorian in victorians: 
        if victorian in author: 
            return True
    return False

In [6]:
vicDF = hasAuthor[hasAuthor.author.apply(isVictorian)]

In [18]:
vicDF = vicDF[vicDF['type'] == 'Text'] # Remove audiobooks

In [19]:
urls = []
for i, row in vicDF.iterrows(): 
    url = None
    # Get utf-8 if possible
    utf = 'text/plain; charset=utf-8'
    plain = 'text/plain'
    asciiText = 'text/plain; charset=us-ascii'
    keys = row.formats.keys()
    formats = row.formats
    if utf in keys: 
        url = formats[utf]
    elif plain in keys: 
        url = formats[plain]
    elif asciiText in keys: 
        url = formats[asciiText]
    # Make sure it's not just a readme
    if 'readme' in url: 
        # Try HTML
        for key in keys: 
            if 'text/html' in key: 
                url = formats[key]
    if url is None: 
        print('Plain text version not found!')
        print(formats)
    vicDF.set_value(i, 'url', url)

In [26]:
set(vicDF['author'])


{'Conrad, Joseph',
 'Eliot, George',
 'Hardy, Thomas',
 'James, Henry',
 'Stevenson, Robert Louis',
 'Ward, Humphry, Mrs.'}

In [25]:
for i, row in vicDF.iterrows(): 
    url = row['url']
    fn = row['url'].split('/')[-1]
    if not os.path.exists(fn): 
        os.system('wget %s' % row['url'])
        time.sleep(1)

In [27]:
len(vicDF['url'])

294

In [28]:
def urlToLocal(url): 
    unSlashed = url.split('/')[-1]
    if unSlashed.endswith('.utf-8'): 
        unSlashed = unSlashed[:-6]
    elif unSlashed.endswith('.zip'): 
        unSlashed = unSlashed.replace('.zip', '.txt')
    return unSlashed

In [31]:
vicDF['localUrl'] = vicDF.url.apply(urlToLocal)

/usr/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [36]:
vicDF.loc[27]

LCC                                                               {PR}
author                                                   Hardy, Thomas
authoryearofbirth                                                 1840
authoryearofdeath                                                 1928
downloads                                                          473
formats              {'text/html': 'http://www.gutenberg.org/ebooks...
id                                                                  27
language                                                          [en]
subjects             {Didactic fiction, Love stories, Wessex (Engla...
title                                       Far from the Madding Crowd
type                                                              Text
url                       http://www.gutenberg.org/ebooks/27.txt.utf-8
localUrl                                                        27.txt
Name: 27, dtype: object

In [33]:
vicDF['localUrl']

27            27.txt
42            42.txt
43          43-0.txt
107          107.txt
110          110.txt
120          120.txt
122        122-0.txt
136          136.txt
143          143.txt
145          145.txt
153          153.txt
176        176-0.txt
177        177-0.txt
178        178-0.txt
179        179-0.txt
208        208-0.txt
209        209-0.txt
210        210-0.txt
211        211-0.txt
219        219-0.txt
220        220-0.txt
224        224-0.txt
280        280-0.txt
281          281.txt
322        322-0.txt
329        329-0.txt
343          343.txt
344          344.txt
372        372-0.txt
373        373-0.txt
            ...     
31484      31484.txt
31557      31557.txt
31809      31809.txt
31916      31916.txt
32649      32649.txt
32939      32939.txt
32954      32954.txt
33325      33325.txt
35546      35546.txt
37424      37424.txt
37425      37425.txt
37627    37627-0.txt
38035      38035.txt
38424      38424.txt
38776    38776-0.txt
40882      40882.txt
42486    4248

In [38]:
for i, row in vicDF.iterrows(): 
    text = open(row.localUrl, errors='ignore').read()
    cleaned = strip_headers(text)
    vicDF.set_value(i, 'text_cleaned', cleaned)

In [39]:
vicDF

,LCC,author,authoryearofbirth,authoryearofdeath,downloads,formats,id,language,subjects,title,type,url,localUrl,text_cleaned
27,{PR},"Hardy, Thomas",1840,1928,473,{'text/html': 'http://www.gutenberg.org/ebooks...,27,[en],"{Didactic fiction, Love stories, Wessex (Engla...",Far from the Madding Crowd,Text,http://www.gutenberg.org/ebooks/27.txt.utf-8,27.txt,\n\n\n\n\n\n\n\n\n\n\n\n\n\nNOTE:\n\nThis edit...
42,{PR},"Stevenson, Robert Louis",1850,1894,4444,{'text/html': 'http://www.gutenberg.org/ebooks...,42,[en],"{Science fiction, Psychological fiction, Londo...",The Strange Case of Dr. Jekyll and Mr. Hyde,Text,http://www.gutenberg.org/ebooks/42.txt.utf-8,42.txt,\n\n\n[Editor's Note: It has been called to ou...
43,{PR},"Stevenson, Robert Louis",1850,1894,1648,{'text/html; charset=utf-8': 'http://www.guten...,43,[en],"{Science fiction, Psychological fiction, Londo...",The Strange Case of Dr. Jekyll and Mr. Hyde,Text,http://www.gutenberg.org/files/43/43-0.txt,43-0.txt,\n\n\n\n\n\n\n\n\n\nTHE STRANGE CASE OF DR. JE...
107,{PR},"Hardy, Thomas",1840,1928,257,{'text/html; charset=iso-8859-1': 'http://www....,107,[en],"{Didactic fiction, Love stories, Wessex (Engla...",Far from the Madding Crowd,Text,http://www.gutenberg.org/ebooks/107.txt.utf-8,107.txt,volunteers and revised by Joseph E. Loewenstei...
110,{PR},"Hardy, Thomas",1840,1928,1932,{'text/html; charset=iso-8859-1': 'http://www....,110,[en],"{Children of the rich -- Fiction, Women househ...",Tess of the d'Urbervilles: A Pure Woman,Text,http://www.gutenberg.org/ebooks/110.txt.utf-8,110.txt,"\n\nE-text transcribed by Steve Menyhert, proo..."
120,"{PZ, PR}","Stevenson, Robert Louis",1850,1894,4193,{'text/plain': 'http://www.gutenberg.org/ebook...,120,[en],"{Treasure Island (Imaginary place) -- Fiction,...",Treasure Island,Text,http://www.gutenberg.org/ebooks/120.txt.utf-8,120.txt,\n\n\n\n\nTREASURE ISLAND\n\nby Robert Louis S...
122,{PR},"Hardy, Thomas",1840,1928,486,{'text/plain; charset=utf-8': 'http://www.gute...,122,[en],"{Mate selection -- Fiction, Mothers and sons -...",The Return of the Native,Text,http://www.gutenberg.org/files/122/122-0.zip,122-0.txt,\n\n\n\n\nTHE RETURN OF THE NATIVE\n\nby Thoma...
136,"{PZ, PR}","Stevenson, Robert Louis",1850,1894,124,{'text/html': 'http://www.gutenberg.org/ebooks...,136,[en],"{Children's poetry, English}",A Child's Garden of Verses,Text,http://www.gutenberg.org/ebooks/136.txt.utf-8,136.txt,\n\n\n\n\n\n\n\n\n\n\n\n\nA Child's Garden of ...
143,{PR},"Hardy, Thomas",1840,1928,515,{'text/html; charset=utf-8': 'http://www.guten...,143,[en],"{Men -- England -- Fiction, Fathers and daught...",The Mayor of Casterbridge,Text,http://www.gutenberg.org/ebooks/143.txt.utf-8,143.txt,\n\n\n\n\n\n\n\nTHE MAYOR OF CASTERBRIDGE\n\nb...
145,{PR},"Eliot, George",1819,1880,2121,{'text/plain; charset=us-ascii': 'http://www.g...,145,[en],"{Bildungsromans, Married people -- Fiction, Di...",Middlemarch,Text,http://www.gutenberg.org/ebooks/145.txt.utf-8,145.txt,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMiddlemarch\n\n\...


In [40]:
vicDF.to_hdf('vic.hdf', 'vic')

/usr/lib/python3.6/site-packages/pandas/core/generic.py:1138: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->['LCC', 'author', 'authoryearofbirth', 'authoryearofdeath', 'downloads', 'formats', 'id', 'language', 'subjects', 'title', 'type', 'url', 'localUrl', 'text_cleaned']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [182]:
james.to_hdf('james.hdf', 'james')

/usr/lib/python3.6/site-packages/pandas/core/generic.py:1138: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->['downloads', 'id', 'language', 'subjects', 'title', 'local_url', 'text_cleaned', 'pub_date']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [134]:
class DBPedia(): 
    """ Looks up metadata about a book from DBPedia, 
    using fuzzy matching on the title and author. """
    def __init__(self, title=None, author=None, resource=None):
        if resource==None and (title==None or author==None): 
            print('Must specify either the resource URI stub, or title and author.')
            raise
        if resource is not None: 
            resource = urllib.parse.unquote(resource)
            self.resourceURI = "http://dbpedia.org/resource/" + resource
        else: 
            response = self.getMeta(title, author)
            parsed = self.parseMeta(response)
            self.resourceURI = self.getResourceURI(parsed)
    
        if self.resourceURI is not None: 
            resourceResponse = self.queryDBP(self.resourceURI)
            resourceParsed = self.parseMeta(resourceResponse)
            self.info = self.getInfo(resourceParsed)
            if self.info is not None: 
                self.pubDate = self.getPubDate()
                self.subjects = self.getSubjects()
                self.literaryGenres = self.getLiteraryGenres()
            else: 
                self._abort()
        else: 
            self._abort()
        
    def _abort(self): 
        self.info = None 
        self.pubDate = None
        self.subjects = None
        self.literaryGenres = None
    
    def getMeta(self, title, author): 
        query = """select distinct ?book
                where {
                  ?book a dbo:Book .
                  ?book dbo:author ?author .
                  ?book dbp:name ?name .
                  FILTER (regex(?name, "%s"))
                  FILTER (regex(?author, "%s"))
                } 
                LIMIT 100""" % (title, author)
        response = requests.get("http://dbpedia.org/sparql", 
                                params=[('query', query), 
                                        ('format', 'application/json')])
        return response

    def parseMeta(self, dbpResponse): 
        try: 
            parsed = json.loads(dbpResponse.text)
        except: 
            print("Couldn't decode JSON: ", dbpResponse.text)
            return None
        return parsed

    def getResourceURI(self, jsonMeta): 
        try: 
            bindings = jsonMeta['results']['bindings']
        except KeyError: 
            print("Couldn't find the appropriate results in the JSON: ", jsonMeta)
            return None
        try: 
            uri = bindings[0]['book']['value']
        except IndexError: 
            print("Couldn't find a resource URI. There is no 0th element in the JSON: ", jsonMeta)
            return None
        return uri
    
    def queryDBP(self, resourceURI):
        query = 'DESCRIBE <%s>' % resourceURI
        params = [('default-graph-uri', 'http://dbpedia.org'), 
                  ('query', query), ('format', 'application/json')]
        r = requests.get("http://dbpedia.org/sparql", params=params)
        return r

    def getInfo(self, jsonData):
        try: 
            data = jsonData[self.resourceURI]
            return data
        except KeyError: 
            print('Strangely, there is no entry in items for this item. Jsondata: ', jsonData)
        
    
    def getPubDate(self): 
        try: 
            releaseDate = self.info['http://dbpedia.org/ontology/releaseDate']
        except KeyError: 
            print('No releaseDate in self.info ontology. Trying property.')
            print('Available info: ', self.info.keys())
            try: 
                releaseDate = self.info['http://dbpedia.org/property/releaseDate']
            except KeyError: 
                print('No releaseDate in self.info property. Giving up.')
                return None
        try: 
            firstRelease = releaseDate[0]
        except IndexError: 
            print('There is no 0th release, apparently: ', releaseDate)
            return None
        try: 
            date = firstRelease['value']
        except KeyError: 
            print('Strangely, no actual date in releaseDate:', releaseDate)
            return None
        return date
    
    def getSubjects(self): 
        subjects = []
        try: 
            subjs = self.info['http://purl.org/dc/terms/subject']
        except KeyError: 
            print('No subject in self.info.')
            return None
        for subj in subjs: 
            subject = subj['value'].split('/')[-1]
            if subject.startswith('Category:'): 
                subject = subject[9:]
            subjects.append(subject)
        return subjects
    
    def getLiteraryGenres(self): 
        literaryGenres = []
        try: 
            genres = self.info['http://dbpedia.org/ontology/literaryGenre']
        except KeyError: 
            print('No literary genre in self.info.')
            return None
        for genre in genres: 
            literaryGenre = genre['value'].split('/')[-1]
            literaryGenres.append(literaryGenre)
        return literaryGenres

In [140]:
rh = DBPedia(title='Roderick Hudson', author='James')
        

No literary genre in self.info.


In [144]:
rh.pubDate

'1875-11-20'

In [145]:
for i, row in james.iterrows(): 
    dbp = DBPedia(title=row.title, author='James')
    if dbp.pubDate is not None: 
        print('Found pubdate: ', dbp.pubDate)
        james.set_value(i, 'pub_date', dbp.pubDate)
    else: 
        print("Didn't find pubdate for: ", row.title)

No literary genre in self.info.
Found pubdate:  1875-11-20

/usr/lib/python3.6/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/lib/python3.6/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s



No releaseDate in self.info ontology. Trying property.
Available info:  dict_keys(['http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'http://www.w3.org/2000/01/rdf-schema#label', 'http://www.w3.org/2000/01/rdf-schema#comment', 'http://www.w3.org/2002/07/owl#sameAs', 'http://www.w3.org/ns/prov#wasDerivedFrom', 'http://dbpedia.org/ontology/abstract', 'http://dbpedia.org/ontology/author', 'http://dbpedia.org/ontology/literaryGenre', 'http://dbpedia.org/ontology/numberOfPages', 'http://xmlns.com/foaf/0.1/name', 'http://dbpedia.org/property/name', 'http://xmlns.com/foaf/0.1/isPrimaryTopicOf', 'http://purl.org/dc/terms/subject', 'http://dbpedia.org/ontology/wikiPageID', 'http://dbpedia.org/ontology/wikiPageRevisionID', 'http://dbpedia.org/ontology/wikiPageExternalLink', 'http://dbpedia.org/property/author', 'http://dbpedia.org/property/genre', 'http://dbpedia.org/property/pages', 'http://dbpedia.org/property/publisher'])
No releaseDate in self.info property. Giving up.
Didn't find pubdate 

Couldn't find a resource URI. There is no 0th element in the JSON:  {'head': {'link': [], 'vars': ['book']}, 'results': {'distinct': False, 'ordered': True, 'bindings': []}}
Didn't find pubdate for:  A Bundle of Letters
Couldn't find a resource URI. There is no 0th element in the JSON:  {'head': {'link': [], 'vars': ['book']}, 'results': {'distinct': False, 'ordered': True, 'bindings': []}}
Didn't find pubdate for:  The Diary of a Man of Fifty
Couldn't find a resource URI. There is no 0th element in the JSON:  {'head': {'link': [], 'vars': ['book']}, 'results': {'distinct': False, 'ordered': True, 'bindings': []}}
Didn't find pubdate for:  The Patagonia
Couldn't find a resource URI. There is no 0th element in the JSON:  {'head': {'link': [], 'vars': ['book']}, 'results': {'distinct': False, 'ordered': True, 'bindings': []}}
Didn't find pubdate for:  The Marriages
Couldn't find a resource URI. There is no 0th element in the JSON:  {'head': {'link': [], 'vars': ['book']}, 'results': {'di

Couldn't find a resource URI. There is no 0th element in the JSON:  {'head': {'link': [], 'vars': ['book']}, 'results': {'distinct': False, 'ordered': True, 'bindings': []}}
Didn't find pubdate for:  The Path Of Duty
Couldn't find a resource URI. There is no 0th element in the JSON:  {'head': {'link': [], 'vars': ['book']}, 'results': {'distinct': False, 'ordered': True, 'bindings': []}}
Didn't find pubdate for:  Four Meetings
Couldn't find a resource URI. There is no 0th element in the JSON:  {'head': {'link': [], 'vars': ['book']}, 'results': {'distinct': False, 'ordered': True, 'bindings': []}}
Didn't find pubdate for:  Embarrassments
Couldn't find a resource URI. There is no 0th element in the JSON:  {'head': {'link': [], 'vars': ['book']}, 'results': {'distinct': False, 'ordered': True, 'bindings': []}}
Didn't find pubdate for:  The Finer Grain
No releaseDate in self.info ontology. Trying property.
Available info:  dict_keys(['http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'htt

Couldn't find a resource URI. There is no 0th element in the JSON:  {'head': {'link': [], 'vars': ['book']}, 'results': {'distinct': False, 'ordered': True, 'bindings': []}}
Didn't find pubdate for:  The Letters of Henry James (Vol. I)
Couldn't find a resource URI. There is no 0th element in the JSON:  {'head': {'link': [], 'vars': ['book']}, 'results': {'distinct': False, 'ordered': True, 'bindings': []}}
Didn't find pubdate for:  The Two Magics: The Turn of the Screw, Covering End


In [150]:
info = DBPedia(title="American", author="James").info

No releaseDate in self.info ontology. Trying property.
Available info:  dict_keys(['http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'http://www.w3.org/2000/01/rdf-schema#label', 'http://www.w3.org/2000/01/rdf-schema#comment', 'http://www.w3.org/2002/07/owl#sameAs', 'http://www.w3.org/ns/prov#wasDerivedFrom', 'http://dbpedia.org/ontology/abstract', 'http://dbpedia.org/ontology/author', 'http://dbpedia.org/ontology/country', 'http://dbpedia.org/ontology/isbn', 'http://dbpedia.org/ontology/numberOfPages', 'http://dbpedia.org/ontology/oclc', 'http://dbpedia.org/ontology/publicationDate', 'http://dbpedia.org/ontology/publisher', 'http://xmlns.com/foaf/0.1/name', 'http://xmlns.com/foaf/0.1/homepage', 'http://dbpedia.org/property/name', 'http://xmlns.com/foaf/0.1/isPrimaryTopicOf', 'http://purl.org/dc/terms/subject', 'http://dbpedia.org/ontology/wikiPageID', 'http://dbpedia.org/ontology/wikiPageRevisionID', 'http://dbpedia.org/ontology/wikiPageExternalLink', 'http://dbpedia.org/property/aut

In [152]:
info['http://dbpedia.org/property/pubDate']

[{'datatype': 'http://www.w3.org/2001/XMLSchema#date',
  'type': 'literal',
  'value': '2012-01-01'}]

In [155]:
james.loc[177].pub_date = '1877-05-05'

In [157]:
james.loc[179].pub_date = '1878-09-18'

In [166]:
james.loc[432].pub_date = '1903-09-24'

In [169]:
james.loc[643].pub_date = '1894'

In [172]:
james.loc[645].pub_date = '1896'

In [174]:
james.loc[898].pub_date = '1888'

In [175]:
james.loc[1032].pub_date = '1891'

In [176]:
james.loc[1144].pub_date = '1898-08-08'

In [180]:
james.loc[1190].pub_date = '1908-12-01'

In [181]:
james

,downloads,id,language,subjects,title,local_url,text_cleaned,pub_date
176,41,176,[en],"{Sculptors -- Fiction, Rome (Italy) -- Fiction...",Roderick Hudson,176-0.txt,\n\n\n\n\nRODERICK HUDSON\n\nby Henry James\n\...,1875-11-20
177,139,177,[en],"{Culture conflict -- Fiction, Separation (Psyc...",The American,177-0.txt,\n\n\nTHE AMERICAN\n\nby Henry James\n\n\n1877...,1877-05-05
178,42,178,[en],"{Americans -- Europe -- Fiction, British -- Eu...",Confidence,178-0.txt,\n\n\n\n\nCONFIDENCE\n\nby Henry James\n\n\n\n...,1880-02-07
179,75,179,[en],"{Upper class -- Fiction, Humorous stories, Eur...",The Europeans,179-0.txt,\n\n\nTHE EUROPEANS\n\nby Henry James\n\n\n\nC...,1878-09-18
208,790,208,[en],"{Europe -- Fiction, Young women -- Fiction, Ps...",Daisy Miller: A Study,208-0.txt,\n\n\n\n\nDAISY MILLER: A STUDY\n\nIN TWO PART...,1879
209,2724,209,[en],"{Children -- Fiction, Ghost stories, Psycholog...",The Turn of the Screw,209-0.txt,\n\n\n\n\nTHE TURN OF THE SCREW\n\nby Henry Ja...,1898-10-13
210,31,210,[en],"{British -- United States -- Fiction, American...",An International Episode,210-0.txt,\n\n\n\n\nAN INTERNATIONAL EPISODE\n\nBy Henry...,NaN
211,199,211,[en],"{Venice (Italy) -- Fiction, Poets -- Fiction, ...",The Aspern Papers,211-0.txt,\n\n\n\n\nTHE ASPERN PAPERS\n\nBy Henry James\...,1888-09-29
432,339,432,[en],"{Paris (France) -- Fiction, Young men -- Ficti...",The Ambassadors,432.txt,"\n\n\n\n\n\n\n\n\nThe Ambassadors,\n\n\nby\n\n...",1903-09-24
642,53,642,[en],"{Memory -- Fiction, Forgiveness -- Fiction, De...",The Altar of the Dead,642-0.txt,\n\nTranscribed from the 1916 Martin Secker ed...,1895-06-18


In [178]:
am = DBPedia(title='Jolly Corner', author='James')

Couldn't find a resource URI. There is no 0th element in the JSON:  {'head': {'link': [], 'vars': ['book']}, 'results': {'distinct': False, 'ordered': True, 'bindings': []}}
